# Ground truth neurons biological features

* author: steeve.laquitaine@epfl.ch

Setup spikeinterf_ env .. 

In [4]:
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import neurom as nm  # ground truth circuit details
import bluepy as bp  # ground truth circuit details
import spikeinterface as si
from neurom.view import plotly_impl
import neurom as nm


# move to PROJECT PATH
PROJ_PATH = "/gpfs/bbp.cscs.ch/project/proj85/home/laquitai/spikebias/"
os.chdir(PROJ_PATH)
from src.nodes.utils import get_config
from src.nodes.load import load_campaign_params

cfg, param_conf = get_config("dense_spont", "probe_2").values()
SORTING_TRUE_PATH = cfg["ground_truth"]["full"]["output"]
BLUECFG = cfg["dataeng"]["blueconfig"]

# PARAMETERS

# SOMA
SOMA = {
    "soma": nm.core.types.NeuriteType.soma,
}
SOMA_FEATURES = [
    "radius",
    "volume",
]

# OTHER NEURITES
NEURITES_FEATURES = [
    "total_length",
    "total_height",
    "total_area",
    "number_of_sections",
    "section_lengths",
]
NEURITES = {
    "apical": nm.core.types.NeuriteType.apical_dendrite,
    "basal": nm.core.types.NeuriteType.basal_dendrite,
    "axon": nm.core.types.NeuriteType.axon,
}

DS1_MORPH = cfg["metadata"]["cell_morphs"]

2024-12-12 17:31:52,972 - root - utils.py - get_config - INFO - Reading experiment config.
2024-12-12 17:31:53,004 - root - utils.py - get_config - INFO - Reading experiment config. - done


### Custom functions

In [5]:
def get_features(morph_paths, feature_set, neurite_set):
    """Get morphometric features for all morphologies passed"""
    features = {"mtype": [], "feature_name": [], "feature_val": [], "loc": []}
    for mtype, paths in tqdm(morph_paths.items(), desc="mtypes"):
        for path in tqdm(
            paths, desc="morphologies", miniters=len(paths) / 100, leave=False
        ):
            morph = nm.load_morphology(path)
            for neurite_name, neurite_id in neurite_set.items():
                for feature in feature_set:
                    features["mtype"].append(mtype)
                    feature_name = (
                        "avg_section_length"
                        if feature == "section_lengths"
                        else feature
                    )
                    features["feature_name"].append(feature_name)
                    feature_val = nm.get(feature, morph, neurite_type=neurite_id)
                    feature_val = (
                        np.mean(feature_val)
                        if feature == "section_lengths"
                        else feature_val
                    )
                    features["feature_val"].append(feature_val)
                    features["loc"].append(neurite_name)
    df = pd.DataFrame.from_dict(features)
    df.drop(
        df.loc[(df["mtype"] == "L4_SSC") & (df["loc"] == "apical")].index, inplace=True
    )  # clear L4_SSC apicals...
    return df


def get_morph_paths(circuit, unit_ids: list):

    # get the units' mtypes and morphologies
    df = circuit.cells.get(unit_ids, properties=["mtype", "morphology"])

    # this creates the morphology path
    # for each unit by prefixing with the
    # common path of all morphologies
    # circuit.config["morphologies"]
    # to the morphology of the
    # units retrieved above
    # e.g., ../fixed_ais_L23PC_20201210/ascii/dend-C220797A-P3_axon-sm110131a1-3_IN.. for unit 12165
    # e.g., ../fixed_ais_L23PC_20201210/ascii/dend-rat_20160908_E3_LH2_cell2_axon-mt for unit 16652
    df["morph_path"] = (
        df["morphology"]
        .apply(lambda row: os.path.join(circuit.config["morphologies"], "%s.asc" % row))
        .astype(str)
    )
    df = df.drop(columns=["mtype", "morphology"])
    return df


def get_radius(circuit, unit_ids):

    # get the file paths of the morphology
    morph_paths = get_morph_paths(circuit, unit_ids)
    morph_paths = morph_paths.values.flatten()

    # get unit soma radius
    soma_radius = []
    for m_i in morph_paths:
        soma_radius.append(nm.load_morphology(m_i).soma.radius)
    return soma_radius

### Soma radius

In [6]:
# load once
circuit = bp.Simulation(BLUECFG).circuit

# get ground truth unit ids
SortingTrue = si.load_extractor(SORTING_TRUE_PATH)

In [7]:
# (1m)
# get volume
radius = get_radius(circuit, SortingTrue.unit_ids)

# unit-test
assert len(radius) == len(SortingTrue.unit_ids)
print(radius)

[7.174989, 8.422994, 7.610226, 12.163964, 6.5572352, 7.9881673, 8.082553, 6.5572352, 8.801092, 10.477856, 6.594521, 8.361107, 9.324264, 8.801092, 8.361107, 6.5572352, 7.275731, 6.5572352, 8.082553, 7.275731, 7.174989, 8.422994, 7.9881673, 7.9881673, 7.9881673, 7.174989, 7.275731, 6.5572352, 7.9881673, 8.422994, 6.594521, 8.422994, 6.5572352, 7.610226, 9.324264, 8.361107, 8.361107, 7.174989, 10.56996, 10.663641, 6.5572352, 10.663641, 6.5572352, 7.047762, 6.5572352, 10.663641, 8.361107, 6.5572352, 7.610226, 8.73887, 8.361107, 6.5572352, 6.472128, 6.9807596, 7.4698386, 7.2470517, 6.896214, 7.368907, 7.368907, 9.613261, 9.613261, 7.2470517, 7.2470517, 7.4698386, 7.4698386, 7.368907, 6.896214, 7.4698386, 7.2470517, 7.2470517, 9.613261, 7.4698386, 7.885879, 12.3498955, 6.434877, 6.434877, 6.434877, 7.2498465, 6.751119, 10.428623, 8.590001, 7.0883703, 7.147172, 8.187446, 6.0981436, 8.358167, 8.266456, 6.182903, 8.731668, 8.187446, 7.0785036, 6.182903, 7.0785036, 7.0785036, 7.1187463, 6.098143

### Neurites features

In [8]:
# 11 min

# get neurite features for each unit
df_all = pd.DataFrame()
for u_i, unit_id in enumerate(SortingTrue.unit_ids):

    # get neurite features
    morph_paths = get_morph_paths(circuit, [unit_id])
    df = get_features(morph_paths, feature_set=NEURITES_FEATURES, neurite_set=NEURITES)
    df = df.apply(lambda x: x.replace("apical", "apical_dendrite"))
    df = df.apply(lambda x: x.replace("basal", "basal_dendrite"))
    df = df.rename({"loc": "neurite_feature"}, axis="columns")
    df["neurite_feature"] += "_" + df["feature_name"]
    df.index = df.neurite_feature.tolist()
    df = df.drop(columns=["feature_name", "mtype", "neurite_feature"])
    df = df.T
    df["morph_path"] = morph_paths.iloc[0][0]
    
    # record
    df_all = pd.concat([df_all, df])

# drop index    
df_all = df_all.reset_index().drop(columns=["index"])

# concatenate with soma radius info
morph_df = pd.DataFrame()
morph_df["unit_id"] = SortingTrue.unit_ids
morph_df["soma_radius"] = radius
df = pd.concat([morph_df, df_all], axis=1)
df.to_parquet(DS1_MORPH)

mtypes: 1it [00:00,  9.09it/s]
mtypes: 1it [00:00,  5.62it/s]
mtypes: 1it [00:00,  7.09it/s]
mtypes: 1it [00:00,  3.43it/s]
mtypes: 1it [00:00,  4.38it/s]
mtypes: 1it [00:00,  2.24it/s]
mtypes: 1it [00:00,  3.54it/s]
mtypes: 1it [00:00,  5.98it/s]
mtypes: 1it [00:00,  2.93it/s]
mtypes: 1it [00:00,  5.49it/s]
mtypes: 1it [00:00,  4.01it/s]
mtypes: 1it [00:00,  3.84it/s]
mtypes: 1it [00:00,  5.80it/s]
mtypes: 1it [00:00,  2.79it/s]
mtypes: 1it [00:00,  4.78it/s]
mtypes: 1it [00:00,  4.90it/s]
mtypes: 1it [00:00,  6.56it/s]
mtypes: 1it [00:00,  6.47it/s]
mtypes: 1it [00:00,  8.33it/s]
mtypes: 1it [00:00,  3.66it/s]
mtypes: 1it [00:00,  7.24it/s]
mtypes: 1it [00:00,  6.84it/s]
mtypes: 1it [00:00,  4.87it/s]
mtypes: 1it [00:00,  7.50it/s]
mtypes: 1it [00:00,  4.84it/s]
mtypes: 1it [00:00,  3.41it/s]
mtypes: 1it [00:00,  7.50it/s]
mtypes: 1it [00:00,  5.11it/s]
mtypes: 1it [00:00,  2.87it/s]
mtypes: 1it [00:00,  4.07it/s]
mtypes: 1it [00:00,  7.01it/s]
mtypes: 1it [00:00, 10.86it/s]
mtypes: 

# References

https://neurom.readthedocs.io/en/latest/features.html#